| [![License: MIT](https://img.shields.io/badge/License-MIT-green.svg)](../LICENSE) | [![Python](https://img.shields.io/badge/Python-3.10+-black.svg)](https://www.python.org/) | [![Jupyter](https://img.shields.io/badge/Jupyter-Notebook-red.svg)](https://jupyter.org/) | [![GeoPandas](https://img.shields.io/badge/Geo-GeoPandas-darkgreen.svg)](https://geopandas.org/) | [![Requests](https://img.shields.io/badge/HTTP-Requests-darkred.svg)](https://docs.python-requests.org/) | [![Pathlib](https://img.shields.io/badge/FS-Pathlib-black.svg)](https://docs.python.org/3/library/pathlib.html) | [![JSON](https://img.shields.io/badge/Data-JSON-grey.svg)](https://www.json.org/) |
|---|---|---|---|---|---|---|


| [![Shapely](https://img.shields.io/badge/Geometry-Shapely-purple.svg)](https://shapely.readthedocs.io/) | [![Fiona](https://img.shields.io/badge/IO-Fiona-orange.svg)](https://fiona.readthedocs.io/) | [![Matplotlib](https://img.shields.io/badge/Plot-Matplotlib-blue.svg)](https://matplotlib.org/) | [![zipfile](https://img.shields.io/badge/Archive-zipfile-lightgrey.svg)](https://docs.python.org/3/library/zipfile.html) | [![GDAL/OGR](https://img.shields.io/badge/IO-GDAL%2FOGR-darkgreen.svg)](https://gdal.org/) |
|---|---|---|---|---|

### **Notebook 2 - Boundary Polygonization and Flood Clipping**

---

**Flood Risk in Lærdal: Official municipality boundary (kommune 4642, Vestland)**
Sources: [https://www.geonorge.no](https://www.geonorge.no) · Dataset search: *“flomsoner”* (Vestland, GML)

### Project overview

**Goal:** Convert the official boundary **lines** from Notebook 1 into a **closed polygon**, validate geometry, compute area, and **clip Vestland flood-zones (flomsoner)** to Lærdal for downstream mapping.
**Method:** Load boundary lines - `polygonize` - geometry checks - area in projected CRS - query/download GML flood layers - reproject - `clip` to Lærdal polygon - export artifacts and quick-look plots.
**Tools:** Python (`pathlib`, `json`, `zipfile`, `requests`, `geopandas`, `fiona`, `shapely` - `ops.polygonize`, `geometry`; `matplotlib.pyplot`)


### Reproducibility - quick reference | Reproduserbarhet - hurtigoversikt

**Inputs (from Notebook 1):**

* `data/processed/laerdal_boundary_lines_2023.geojson` *(EPSG:25832)*
* `data/processed/laerdal_bbox_centroid.json`

** Outputs (this notebook):**
* `data/raw/Vestland_Flomsoner_GML.zip` *(GeoNorge download)*
* `data/raw/vestland_flomsoner.gml` *(extracted)*
* `data/processed/laerdal_boundary_polygon_2023.geojson` *(validated polygon, EPSG:25832)*
* `data/processed/laerdal_floodzones_clipped.geojson` *(all available return-periods, EPSG:4326 for web maps)*
* `results/laerdal_boundary_polygon_preview.png`

**Parameters (this notebook):**

* **Kommune number** = 4642 (Lærdal)
* **Projection for area calculation** = EPSG:25832
* **Projection for visualization/web export** = EPSG:4326
* **GeoNorge API search**: `query = "flomsoner"`
* **Region/dataset hint**: *Vestland flomsoner (GML)*

> **Notes:**
> • This notebook **consumes** Notebook 1 outputs; it does **not** re-download the boundary dataset.

In [ ]:
# (EN) Reproducibility parameters and paths
# (NO) Reproduserbarhetsparametere og stier

MUNICIPALITY = "Lærdal"
KOMMUNE_NR = "4642"
YEAR = "2023"

# Coordinate Reference Systems
CRS_EPSG = "EPSG:25832"  # Projected CRS for spatial analysis (UTM zone 32N)
CRS_WGS84 = "EPSG:4326"  # Geographic CRS for visualization (lat/lon)

# Project directories (relative to notebooks)
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
RESULT_DIR = PROJECT_ROOT / "results" 

# Ensure output folders exist
for folder in [RAW_DIR, PROCESSED_DIR, RESULT_DIR]:
    folder.mkdir(parents=True, exist_ok=True) 

# Define standard output paths
BOUNDARY_LINES_GEOJSON = PROCESSED_DIR / f"laerdal_boundary_lines_{YEAR}.geojson"
BOUNDARY_POLY_GEOJSON = PROCESSED_DIR / f"laerdal_polygon_{YEAR}.geojson"
FLOOD_ZONE_GML = RAW_DIR / f"Samfunnssikkerhet_46_Vestland_25832_Flomsoner_GML.gml"
BOUNDARY_POLY_PNG = RESULT_DIR / f"laerdal_boundary_polygon_{YEAR}.png" 
FLOOD_ZONE_GML_LÆRDAL = PROCESSED_DIR/ f"laerdal_floodzone_{YEAR}.geojson" 

In [ ]:
# (EN) Import required libraries and verify data/result folders
# (NO) Importerer nødvendige biblioteker og bekrefter mapper for data/resultater

# libraries
import json
import zipfile
import geopandas as gpd
import requests
import matplotlib.pyplot as plt
from shapely.geometry import LineString, MultiLineString
from shapely.ops import polygonize

# Confirm folder setup
print(f"  • RAW_DIR exists:        {RAW_DIR.exists()}")
print(f"  • PROCESSED_DIR exists:  {PROCESSED_DIR.exists()}")
print(f"  • RESULT_DIR exists:    {RESULT_DIR.exists()}") 

#### Polygonize Lærdal kommune boundary – Konverter til polygon

**(EN):** Convert the Lærdal municipality boundary from line segments (`LineString`) into a closed polygon (`Polygon`) to enable area calculations, clipping, and mapping.
**(NO):** Konverter kommunegrensen for Lærdal fra linjesegmenter (`LineString`) til et lukket polygon (`Polygon`) for å muliggjøre arealberegninger, utklipping og kartlegging.


In [ ]:
# (EN) Converts Lærdal boundary from LineString to Polygon. This enables area calculation, clipping, and visualizations.
# (NO)  Konverterer kommunegrensen for Lærdal fra LineString til Polygon. Dette muliggjør arealberegning, utklipping og visualisering.

# Load original boundary
gdf = gpd.read_file(BOUNDARY_LINES_GEOJSON)

# Combine all LineStrings into a single MultiLineString
multi = gdf.geometry.union_all()

# Ensure correct type for polygonization
if isinstance(multi, LineString):
    multi = MultiLineString([multi])

# Convert boundary lines into one or more polygons
polygons = list(polygonize(multi))

# Create GeoDataFrame with polygonized boundary
gdf_poly = gpd.GeoDataFrame(geometry=polygons, crs=gdf.crs)

# Confirm geometry type
print("Geometry types:", gdf_poly.geometry.type.unique())

# Save the polygon as GeoJSON
gdf_poly.to_file(BOUNDARY_POLY_GEOJSON, driver="GeoJSON")
print(f"Saved polygonized boundary to: {BOUNDARY_POLY_GEOJSON}")

In [ ]:
# (EN) Visualize the polygon geometry to confirm successful conversion, and  estimate the total area in km²
# (NO) Visualiser polygon-geometrien for å bekrefte vellykket konvertering. Beregn totalarealet i km²

# Load file 
gpd.read_file(BOUNDARY_POLY_GEOJSON)
                          
ax = gdf_poly.plot(edgecolor="black", figsize=(6, 6))
ax.set_title("Lærdal kommune - Polygonized boundary")
ax.set_axis_off()

plt.savefig(BOUNDARY_POLY_PNG, dpi=300, bbox_inches="tight", pad_inches=0.1)
plt.show()

print(f"Saved figure to: {BOUNDARY_POLY_PNG}")

# Estimate the total area in km²
area_km2 = gdf_poly.geometry.area.sum() / 1e6
print(f"Estimated area of Lærdal kommune: {area_km2:.2f} km²")

_____
#### Dataset Metadata Search – Flood Zones 

This section displays metadata retrieved from [kartkatalog.geonorge.no](https://kartkatalog.geonorge.no) for datasets related to **"flomsoner"** (flood zones).

**Crucial Note on Data Access:**

Not all listed datasets provide files directly available for download via an API. The next step in this notebook will attempt automated downloading, but in many cases, manual intervention is required.

To obtain the necessary data manually:

1.  Open the link provided in the `ShowDetailsUrl` column of the desired dataset.
2.  Scroll down to the **“Nedlastning”** (Download) section.
3.  Manually select and copy the ZIP download link. This data is usually grouped by county (e.g., “Vestland” for Lærdal).

***

In [ ]:
# (EN) Search public flood datasets from GeoNorge
# Query the GeoNorge catalogue for datasets related to flood zones (keyword: "flomsoner").
# (NO) Søk etter offentlige flomdatasett fra GeoNorge
# Spørr GeoNorge-katalogen etter datasett relatert til flomsoner (søkeord: "flomsoner").

# Define search query and URL
query = "flomsoner"
url = f"https://kartkatalog.geonorge.no/api/search?text={query}"

# Send request and parse response
response = requests.get(url)
response.raise_for_status()
data = response.json()

# Access search results
results = data.get("Results", [])
print(f"Found {len(results)} datasets for query '{query}'\n")

In [ ]:
# (EN) Display titles of flood related datasets
# (NO) Vis titler for flomrelaterte datasett

max_results = 10
for i, item in enumerate(results[:max_results], 1):
    print(f"{i}. {item['Title']}")

# Inspect the first search hit and print key fields (UUID, details page, organization) to locate the exact dataset for download.
for i, item in enumerate(results[:1], 1):
    print(" - UUID:", item['Uuid'])
    print(" - Details page:", item['ShowDetailsUrl'])
    print(" - Organization:", item['Organization'])
    print()

In [ ]:
#(EN) Unzip the flood zone dataset manually downloaded from GeoNorge for Vestland region and inspect its contents
#(NO) Pakk ut flomsonedataene manuelt lastet ned fra GeoNorge for Vestland fylke, og inspiser innholdet

zip_path = RAW_DIR / "Vestland_Flomsoner_GML.zip"  
extract_path = RAW_DIR  
extract_path.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    print("Extracted files:")
    for f in extract_path.glob("*"):
        print(" -", f.name)

In [ ]:
# (EN) List all available layers in the GML file
# (NO) List alle tilgjengelige lag (layers) i GML-filen

import fiona

# Display available layers
layers = fiona.listlayers(FLOOD_ZONE_GML)
print("Layers found in the GML file:")
for name in layers:
    print("-", name)

In [ ]:
#(EN) Load the Lærdal municipality boundary and national flood zone dataset.
#(NO) Last inn kommunegrensen for Lærdal og nasjonalt datasett for flomsoner.
 
# Load boundary
boundary = gpd.read_file(BOUNDARY_POLY_GEOJSON)

# Load flood zones 
gdf_flood = gpd.read_file(FLOOD_ZONE_GML,layer="FlomAreal")

# Preview the data
print("Boundary CRS:", boundary.crs)
print("Flood zones CRS:", gdf_flood.crs)
display(gdf_flood.head())

In [ ]:
#(EN) Preview and verify data
#(NO) Forhåndsvis og kontroller dataene

print(" - Features boundary:", len(boundary))
print(" - Features flood:", len(gdf_flood))
print(" - Geometry types:", gdf_flood.geometry.type.unique())
print(" - Valid geometries?:", gdf_flood.is_valid.all())
print(" - Any empty geometries?:", gdf_flood.is_empty.any())

In [ ]:
# (EN) Clip the flood zones
# (NO) Klipp flomsonene

# Clip the flood GeoDataFrame using the boundary polygon(s)
flom_laerdal = gpd.clip(gdf_flood, boundary)
print("Clipped flood zones ('flom_laerdal') successfully created.")

# Save result for later use
try:
    # Save the resulting GeoDataFrame to the specified file path as GeoJSON
    flom_laerdal.to_file(FLOOD_ZONE_GML_LÆRDAL, driver="GeoJSON")

    # Confirmation messages
    print(f"\n - Flood zones successfully saved to: {FLOOD_ZONE_GML_LÆRDAL}")
    print(" - Flood zone in Lærdal saved.")

except Exception as e:
    # Handle any errors during the saving process
    print(f"\n Error saving the file: {e}")

#### Boundary Polygonization and Flood Clipping Completed | Polygonisering og klipping fullført

**EN:** Phase 2 (2023) is complete:
- Polygonized Lærdal’s boundary from line segments and saved as `laerdal_boundary_polygon_2023.geojson`.
- Verified geometry type and computed area in a metric CRS (ETRS89 / UTM 32N, EPSG:25832).
- Inspected GeoNorge “flomsoner” metadata, unzipped the **Vestland** package, and listed GML layers.
- Selected the `FlomAreal` layer (flood polygons) and **clipped** it to the Lærdal boundary.
- Saved results to `floodzones_laerdal_2023.geojson` for reuse in mapping and exposure analysis.

**NO:** Fase 2 (2023) er fullført:
- Polygoniserte Lærdals kommunegrense fra linjesegmenter og lagret som `laerdal_boundary_polygon_2023.geojson`.
- Verifiserte geometri og beregnet areal i metrisk CRS (ETRS89 / UTM 32N, EPSG:25832).
- Gikk gjennom GeoNorge-metadata for «flomsoner», pakket ut **Vestland**-pakken og listet GML-lag.
- Valgte laget `FlomAreal` (flompolygoner) og **klippet** det til Lærdal-grensen.
- Lagret resultater til `floodzones_laerdal_2023.geojson` for videre kartlegging og eksponeringsanalyser.

_____

**Navigation Links**
  
- [Notebook 1 - Data Collection, Inspection and Preparation](./01_prepare_boundary.ipynb)  